In [ ]:
from oneatlas import OneAtlasClient  # custom class in this repo
import json
import geopandas as gpd
from pathlib import Path

# Process overview
- Convert input aoi layer into geojson polygons
- For each aoi polygon feature:
  - Search API for available images
  - Plot the quicklook for each image found for that feature
  - Place an order for the chosen image clipped to the feature
  - Download the order once completed

In [ ]:
# Read local config.json to get api key and directory for outputs
with open("config.json", "r") as file:
    config = json.load(file)

api_key = config["api_key"]

client = OneAtlasClient(api_key=api_key)

output_folder = Path(config["output_dir"])

In [ ]:
# function to convert polygon gdf to bounding box list
aoi_gdf = gpd.read_file("inputs/test_quarries.shp")

LAYER_NAME = "test"  # prefix name for the clipped image downloads

In [ ]:
# If necessary buffer the image
aoi_gdf["geometry"] = aoi_gdf.buffer(1000)

In [ ]:
def aoi_gdf_to_search_geojson(aoi_gdf, uid_column=None):
    """Convert geodataframe to geojson ensuring unique ids per feature"""

    if uid_column is None:

        uid_column = "id"

        aoi_gdf[uid_column] = aoi_gdf.index

    else:

        if not aoi_gdf[uid_column].is_unique:

            raise ValueError(
                f"Values in '{uid_column}' are not unique. Consider using default uid_column=None to add uids."
            )

    print(
        f"uid_column '{uid_column}' values {', '.join(str(i) for i in aoi_gdf[uid_column].tolist())}"
    )

    json_str = aoi_gdf[[uid_column, "geometry"]].to_crs(epsg=4326).to_json(drop_id=True)

    return json.loads(json_str), aoi_gdf[uid_column].tolist()

In [ ]:
# convert gdf to geojson and get the id value list to know ids to search for
search_geojson, id_vals = aoi_gdf_to_search_geojson(aoi_gdf)

In [ ]:
def get_feature_by_id(geojson, id_vals, search_id, uid_column="id"):
    """Filter geojson features to just one specified id"""

    if search_id not in id_vals:

        raise ValueError(f"id {search_id} not in list of id values")

    for feature in geojson["features"]:

        if feature["properties"][uid_column] == search_id:
            return feature

    return None

In [ ]:
# Set the search id. Must be in the id_vals list created above.
SEARCH_ID = 1
# extract the geojson feature for that id
search_feature = get_feature_by_id(search_geojson, id_vals, SEARCH_ID)

Search options are described [here](https://www.geoapi-airbusds.com/guides/oneatlas-data/g-search/)

In [ ]:
# create search json - geometry is the search feature geometry
img_search_json = {
    "cloudCover": "[0,30]",
    "incidenceAngle": "[0,40]",
    "processingLevel": "SENSOR",
    "relation": "contains",
    "geometry": search_feature["geometry"],
    "constellation": "PHR",
}

# make the request
results = client.search(img_search_json)

# extract relevant values from the result and store in client instance


client.extract_results(results)

In [ ]:
# Keep running this to show each search result image quicklook in turn
client.show_result()

See the order options [here](https://www.geoapi-airbusds.com/guides/oneatlas-data/g-order-product/)

In [ ]:
# create the order specification and initially just quote the price
order_body = {
    "kind": "order.data.product",
    "products": [
        {
            "productType": "multiSpectral",

            "radiometricProcessing": "REFLECTANCE",
            "imageFormat": "image/geotiff",
            "crsCode": "urn:ogc:def:crs:EPSG::4326",
            "id": client.current_image,  # current client.show_result() image
            "aoi": search_feature["geometry"],
        }
    ],
}



client.get_price(order_body)["price"]

In [ ]:
# Uncomment line below to place the order above (WARNING: account will be charged)
# client.create_order(order_body)

In [ ]:
# See current and previous orders - select which want
orders = client.list_orders(kind="order.data.product", status="delivered")
orders

In [ ]:
ORDER_INDEX = 0  # Set as required. 0-indexed item number from the orders list
order = orders["items"][ORDER_INDEX]

# The config.json in the repo specifies the output_folder
output_file = output_folder / f"{LAYER_NAME}_{SEARCH_ID}.zip"

# Download the order to specified zip file
client.download_order_to_file(order, output_file)